In [1]:
import numpy as np
from scipy.spatial import distance_matrix

In [2]:
def read_landmarks(filename,only_2d=True):
    manual_landmark_list = []
    lines = []
    with open(filename, 'r') as csb:
        lines = csb.readlines()
    number = 0

    # for each manual landmark
    for line in lines:
        #print line
        if line.startswith("@1"):
            number = 1
            continue
        if number == 1 and line.isspace() == False:
            #print line.split()
            pt = list(map(float,line.split()))
            #pt = [pt[0]*self.axis_directions[0],pt[1]*self.axis_directions[1],pt[2]*self.axis_directions[2]]
            manual_landmark_list.append([pt[0],pt[1]])

    return manual_landmark_list

In [3]:
def write_landmarks(landmarks,filename):
    
    with open(filename, 'w') as file:
        file.write("# Avizo 3D ASCII 2.0 \n")
        file.write("define Markers ")
        file.write("{}".format(len(landmarks)))
        file.write("\n")
        file.write("Parameters {\n")
        file.write("    NumSets 1,\n")
        file.write("    ContentType \"LandmarkSet\"\n")
        file.write("}\n")

        file.write("Markers { float[3] Coordinates } @1\n")

        file.write("# Data section follows\n")
        file.write("@1\n")

        if len(landmarks)!=0:
            for landmark in landmarks:
                ##print(landmark[0][0])
                file.write("{}".format(landmark[0]))
                file.write(" ")
                file.write("{}".format(landmark[1]))
                file.write(" ")
                if len(landmark) > 2:
                    file.write("{}".format(landmark[2]))
                else:
                    file.write("{}".format(0))
                file.write("\n")

In [19]:
#manual_landmarks_file = '/nas1/Data_aman/00_Rabies/MG48_3Day_bs/01_Data/00_Landmark/000_manual/00_original/S051_Landmarks.landmarkAscii'
#auto_landmarks_file_full = '/nas1/Data_Mythreya/RabiesCountingProject/22june_MG48_S51_trinary.landmarkAscii' 
manual_landmarks_file = '//ibs9010/current_data/Data_Keshav/image/Classification_model/manual_landmark_files/S051_Landmarks.landmarkAscii'
auto_landmarks_file_full = "//ibs9010/current_data/Data_Keshav/image/Classification_model/landmarks/21july_MG48_S051_binary.landmarkAscii"


#auto_landmarks_file_partial = '/nas1/Data_Mythreya/RabiesCountingProject/MG48_S51-26may-PN.landmarkAscii'
error_resolution = 25 # radius around manual landmark where we compare , in microns
output_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/LMfiles/'
exp = 'S051'

manual_landmarks = read_landmarks(manual_landmarks_file)
auto_landmarks_full = read_landmarks(auto_landmarks_file_full)
#auto_landmarks_partial = read_landmarks(auto_landmarks_file_partial)

auto_landmarks_all = []
for pt in auto_landmarks_full:
    auto_landmarks_all.append(pt)
# for pt in auto_landmarks_partial:
#     auto_landmarks_all.append(pt)
    
manual_np = np.array(manual_landmarks)
auto_np = np.array(auto_landmarks_all)


In [26]:
confusion_mat = (distance_matrix(manual_np,auto_np) < error_resolution)
(confusion_mat.sum(axis=1)>0)

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True, False,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True, False, False,  True, False, False,  True, False, False,
        True,  True, False,  True,  True, False, False, False, False,
        True,  True,  True,  True, False,  True,  True,  True, False,
        True, False,

In [27]:
TP = (confusion_mat.sum(axis=1)>0).sum()
FN = (confusion_mat.sum(axis=1)==0).sum()
FP = (confusion_mat.sum(axis=0)==0).sum()

TP_per = (TP/len(manual_landmarks))*100
FN_per = FN/len(manual_landmarks)
FP_per = FP/len(manual_landmarks)

TP_locations = np.where(confusion_mat.sum(axis=1))
FN_locations = np.where(confusion_mat.sum(axis=1)==0)
FP_locations = np.where(confusion_mat.sum(axis=0)==0)

write_landmarks(manual_np[TP_locations],output_path+exp+'_Matches(TP).landmarksAscii')
write_landmarks(manual_np[FN_locations],output_path+exp+'_Misses(FN).landmarksAscii')
write_landmarks(auto_np[FP_locations],output_path+exp+'_FP.landmarksAscii')

In [29]:
TP,FN,FP

(122, 55, 96)

In [30]:
TP_per,FN_per,FP_per

(68.92655367231639, 0.3107344632768362, 0.5423728813559322)